In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# fixes weird issue with pyLDAvis (?) warnings

import pandas as pd
import numpy as np
import pickle
import operator
import re
import gc
import gensim
# from gensim.similarities import WmdSimilarity

import pyLDAvis
import pyLDAvis.gensim

# wtf
warnings.filterwarnings("ignore", category=DeprecationWarning)

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from tw_dataset.dbmodels import *
from random import sample
import json

Switching to API Credentials #8


In [2]:
# s = open_session()
# all_tweets_text_es = [t.text for t in s.query(Tweet).all() if t.lang == 'es']

# with open('all_tweets_text_es.json', 'w') as f:
#     json.dump(all_tweets_text_es, f)

In [3]:
with open('all_tweets_text_es.json') as f:
    all_tweets_text_es = json.load(f)
len(all_tweets_text_es)    

1636480

In [4]:
# tweets = sample(all_tweets_text_es, 5000)
tweets = all_tweets_text_es

# OK, let's finally do some topic modelling

In [5]:
from tokenizer import tokenize, spanish_stopwords

In [6]:
def preprocess(doc):
    pre_doc = doc
        
    # remove URLs
    pre_doc = re.sub(
        r"https?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
        " ", pre_doc)
        
    # find and split hashtags
    # very simple splitting (TODO: come up with something wittier)
    # split on capital letters, but only if hashtag longer than 5
    # → conditional is to avoid splitting abbr. like "IoT" or "NSA"
    pre_doc = re.sub(r"(?:^|\s)[＃#]{1}(\w+)", 
            lambda s: re.sub(r"([A-Z])", r" \1", s.group(0)) if len(s.group(0)) > 5 else s.group(0), 
            pre_doc)
    pre_doc = re.sub(r"＃|#", " ", pre_doc)
    
    # lowercase everything
    pre_doc = pre_doc.lower()
        
    # remove bullshit
    pre_doc = re.sub(r"\@|\'|\"|\\|…|\/|\-|\||\(|\)|\.|\,|\!|\?|\:|\;|“|”|’|—", " ", pre_doc)
    
    # normalize whitespaces
    pre_doc = re.sub(r"\s+", " ", pre_doc)
    pre_doc = re.sub(r"(^\s)|(\s$)", "", pre_doc)
    
    return pre_doc

In [7]:
class get_docs(object):
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for doc in self.corpus:
            tokens = tokenize(preprocess(doc))
            yield tokens

In [8]:
# dictionary = gensim.corpora.Dictionary(get_docs(tweets))
# dictionary.filter_extremes(no_below=100, no_above=0.1, keep_n=None)
# dictionary.save("tweets_es.dict")

In [9]:
dictionary = gensim.corpora.Dictionary.load("tweets_es.dict")

2017-04-02 12:56:30,283 : INFO : loading Dictionary object from tweets_es.dict


In [10]:
# bow = [dictionary.doc2bow(doc) for doc in get_docs(tweets)]
# with open('tweets_es_bow.pickle', 'wb') as f:
#     pickle.dump(bow, f)

In [11]:
with open('tweets_es_bow.pickle', 'rb') as f:
    bow = pickle.load(f)

In [12]:
n_topics = 25
iters = 40
passes = 10

In [17]:
# gensim.models.ldamodel.LdaModel?

In [13]:
# model = gensim.models.ldamodel.LdaModel(
model = gensim.models.LdaMulticore(
        corpus=bow,
        id2word=dictionary,
        num_topics=n_topics,
        iterations=iters,
        alpha=0.001,
        passes=passes,
        chunksize=10000,
        workers=4
#         distributed=True
)
model.save("tweets_es_%dtopics.lda" % n_topics)

2017-04-02 13:01:06,867 : INFO : using serial LDA version on this node
2017-04-02 13:01:14,955 : INFO : running online LDA training, 25 topics, 10 passes over the supplied corpus of 1636480 documents, updating every 40000 documents, evaluating every ~400000 documents, iterating 40x with a convergence threshold of 0.001000
2017-04-02 13:01:14,961 : INFO : training LDA model using 4 processes
2017-04-02 13:01:15,570 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #10000/1636480, outstanding queue size 1
2017-04-02 13:01:15,996 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #20000/1636480, outstanding queue size 2
2017-04-02 13:01:16,494 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #30000/1636480, outstanding queue size 3
2017-04-02 13:01:16,899 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #40000/1636480, outstanding queue size 4
2017-04-02 13:01:17,219 : INFO : PROGRESS: pass 0, dispatched chunk #4 = documents up 

In [14]:
model = gensim.models.LdaModel.load("tweets_es_%dtopics.lda" % n_topics)

2017-04-02 15:03:12,216 : INFO : loading LdaModel object from tweets_es_25topics.lda
2017-04-02 15:03:12,233 : INFO : setting ignored attribute state to None
2017-04-02 15:03:12,234 : INFO : setting ignored attribute dispatcher to None
2017-04-02 15:03:12,235 : INFO : loading LdaModel object from tweets_es_25topics.lda.state


In [15]:
viz = pyLDAvis.gensim.prepare(model, [v for v in bow], model.id2word)
pickle.dump(viz, open("tweets_es_%d.viz" % n_topics,'wb'))

In [16]:
viz = pickle.load(open("tweets_es_%d.viz" % n_topics,'rb'))
pyLDAvis.display(viz)